# Imports, install and mount

In [9]:
import sys
sys.path.append("/..")
from play_with_complex import *

import random
import torch
import torch.optim as optim
import torch_geometric
from torch_geometric.nn import ComplEx
from torch_geometric.data import Data
import pandas as pd
from torch_geometric.loader import DataLoader
from nxontology.imports import from_file
import pickle
import wandb
import torch.nn.functional as F
import numpy as np


ImportError: cannot import name 'all' from 'play_with_complex' (/home/ebutz/ESL2024/code/utils/play_with_complex.py)

# Settings 


In [2]:
# Datas
mapped_iric_path = '/home/ebutz/ESL2024/data/Os_GO_altailed_mapped_iric.tsv'
datasets_save_path = '/home/elliot/Documents/ESL2024/data/mapping_datasets_and_model_for_genes_to_phenotypes_iric/test_dataset_'

val_path = datasets_save_path + 'VAL' +  '.pickle'
test_path = datasets_save_path + 'TEST' +  '.pickle'
train_path = datasets_save_path + 'TRAIN' +  '.pickle'

# Model to load :
hidden_channels = 220

params_save_name = f"PARAMS_ComplEx_6_times_{hidden_channels}_HC_{500}_epochs_{4096}_BS_on_full_Os_GO"
model_parameters_path = "/home/elliot/Documents/ESL2024/data/mapping_datasets_and_model_for_genes_to_phenotypes_iric/"+params_save_name


# Model to re-train :
hidden_channels = 220
batch_size = 4096
epochs = 500
lin_factor = 0.5

params_save_name = f"PARAMS_ComplEx_6_times_{hidden_channels}_HC_{epochs}_epochs_{batch_size}_BS_on_full_Os_GO"
model_parameters_path = "/home/elliot/Documents/ESL2024/data/mapping_datasets_and_model_for_genes_to_phenotypes_iric/"+params_save_name

# Ontology
url = "/home/elliot/Documents/ESL2024/data/go-basic.json.gz"

# Device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = 'cpu' # Tip : Use cpu for debugging
print(device)

cuda


In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
a = torch.Tensor([0,1])
print(a.is_cuda)
a = a.to(device)
print(a.is_cuda)

cuda
False
True


# data loading


In [4]:
mapped_iric = pd.read_csv(mapped_iric_path, sep = '\t', names=['subject', 'predicate', 'object', 'mapped_subject', 'mapped_predicate', 'mapped_object','mapped_alt_tails'])
display(mapped_iric)

GO_to_map = mapped_iric.set_index('object')['mapped_object'].to_dict()
map_to_GO = {value: key for key, value in GO_to_map.items()}


# Checking dict :
looks_ok: bool = True
print(len(list(mapped_iric['object'])))
for i in range(len(list(mapped_iric['object']))):
    if GO_to_map[mapped_iric['object'][i]]!=mapped_iric['mapped_object'][i]:
        looks_ok = False
    
print('Dict looks ok :', looks_ok)
print(map_to_GO)
print(GO_to_map)

,subject,predicate,object,mapped_subject,mapped_predicate,mapped_object,mapped_alt_tails


0
Dict looks ok : True
{}
{}


In [5]:
from tqdm import tqdm
tqdm.pandas()

mapped_iric['mapped_alt_tails'] = mapped_iric.progress_apply(
                                             lambda row: 
                                             mapped_iric.loc[(mapped_iric['mapped_subject'] == row['mapped_subject']) & 
                                                             (mapped_iric['mapped_predicate'] == row['mapped_predicate']) & 
                                                             (mapped_iric['mapped_object'] != row['mapped_object']), 
                                                             'mapped_object'].values, 
                                             axis=1)
display(mapped_iric)

1it [00:00, 257.54it/s]


,subject,predicate,object,mapped_subject,mapped_predicate,mapped_object,mapped_alt_tails


In [6]:
mapped_alt_tails = {}

for index, row in mapped_iric.iterrows():
    key = (row['mapped_subject'], row['mapped_predicate'])
    if key not in mapped_alt_tails:
        mapped_alt_tails[key] = set()
    mapped_alt_tails[key].update(set(row['mapped_alt_tails']))

print(mapped_alt_tails)

for key, value in mapped_alt_tails.items():
    mapped_alt_tails[key]=np.array(list(value))

print(mapped_alt_tails[(30291, 0)])

{}


KeyError: (30291, 0)

In [ ]:
val_data = torch.load(val_path)
test_data = torch.load(test_path)
train_data = torch.load(train_path)

print("Datatsets look OK ? (val, train, test) :",
val_data.validate(),
test_data.validate(),
train_data.validate())

print(val_data)
print(test_data)
print(train_data)

Datatsets look OK ? (val, train, test) : True True True
Data(edge_index=[2, 135400], edge_attr=[135400], num_nodes=30396, edge_label=[33848], edge_label_index=[2, 33848])
Data(edge_index=[2, 152324], edge_attr=[152324], num_nodes=30396, edge_label=[33848], edge_label_index=[2, 33848])
Data(edge_index=[2, 135400], edge_attr=[135400], num_nodes=30396, edge_label=[135400], edge_label_index=[2, 135400])


In [ ]:
subs = set(list(mapped_iric['mapped_subject']))
objs = set(list(mapped_iric['mapped_object']))

heads = set(train_data.edge_index[0].tolist())
tails = set(train_data.edge_index[1].tolist())

print('Number of heads in dataset  :', len(heads))
print('difference heads - subjects :', len(heads-subs))
print('difference heads - object   :', len(heads-objs))

print('Numer of tails in dataset   :', len(tails))
print('Difference tails - subjects :', len(tails-subs))
print('Differece tails - objects   :', len(tails-objs))

Number of heads in dataset  : 22648
difference heads - subjects : 0
difference heads - object   : 22648
Numer of tails in dataset   : 6557
Difference tails - subjects : 6557
Differece tails - objects   : 0


In [ ]:
# Here I create a batch on wich i will test my losses. I need a loader and a model to create it.

complex_model = ComplEx(
    num_nodes=train_data.num_nodes,
    num_relations = train_data.edge_index.size()[1],
    hidden_channels=hidden_channels,
).to(device)

loader = complex_model.loader(
    head_index = train_data.edge_index[0],
    tail_index = train_data.edge_index[1],
    rel_type = train_data.edge_attr,
    batch_size=batch_size,
    shuffle=False,)

batchy = next(iter(loader))
hi, rt, ti = batchy[0], batchy[1], batchy[2]
print('\n',hi,'\n',rt,'\n', ti)


 tensor([12043,  2912, 16271,  ...,   478,  9393, 27290]) 
 tensor([0, 0, 0,  ..., 0, 0, 0]) 
 tensor([16231,  1781, 13382,  ..., 18714,  3859,   157])


# Defining LinLoss() functions

In [ ]:
nxo = from_file('/home/elliot/Documents/ESL2024/data/go-basic.json.gz')
nxo.freeze()
print(nxo)

In [ ]:
def lin_sim_on_mapped_terms(mapped_term1, mapped_term2):
    term1 = map_to_GO[mapped_term1]
    term2 = map_to_GO[mapped_term2]
    if (term1 in nxo.graph._node and term2 in nxo.graph._node):
        sim = nxo.similarity(term1, term2).lin 
        return sim
    else:
        return 0  


def best_lim_sim_for_triple(head, rel, tail)-> torch.Tensor:
    max_lin_sim=0
    for alt_tail in mapped_alt_tails[(head, rel)]:

        if (map_to_GO[tail] in nxo.graph._node
            and
            map_to_GO[alt_tail] in nxo.graph._node):
                
                sim = nxo.similarity(map_to_GO[tail], map_to_GO[alt_tail]).lin # Pourrait être amélioré : actuellement, on calcule plein de similarités différentes.
                
                if max_lin_sim < sim < 1:
                    max_lin_sim = sim
    
    return max_lin_sim


def best_lin_sims_for_batch(head_index:torch.Tensor, rel_type:torch.Tensor, tail_index:torch.Tensor):

    batch = pd.DataFrame(torch.transpose(torch.stack((head_index,rel_type,tail_index)),
                                         0,1)
                        )
    
    return torch.Tensor(batch.apply(lambda row : best_lim_sim_for_triple(head=row[0],
                                                                         rel=row[1],
                                                                         tail=row[2]),
                                    axis=1))

def lin_sims_for_batch(term1: torch.Tensor, term2: torch.Tensor)->torch.Tensor:

    batch = pd.DataFrame(torch.transpose(torch.stack((term1, term2)),
                                         0,1)
                        )
    
    display(batch)
    
    return torch.Tensor(batch.apply(lambda row : best_lim_sim_for_triple(head=row[0],
                                                                         rel=row[1],
                                                                         tail=row[2]),
                                    axis=1))

In [ ]:
def shuffle_tensor(t: torch.Tensor):
    '''
    Shuffles elments of a tensor.
    WARNING :
    shuffle_tensor(torch.tensor([[0,1,2,3,4,5],[6,7,8,9,0,1]]))
    returns :
    tensor([[0, 1, 2, 3, 4, 5],  OR tensor([[6, 7, 8, 9, 0, 1],
            [6, 7, 8, 9, 0, 1]])            [0, 1, 2, 3, 4, 5]])
    '''
    idx = torch.randperm(t.shape[0])
    return t[idx].view(t.size())

shuffle_tensor(torch.tensor([[0,1,2,3,4,5],[6,7,8,9,0,1]]))

tensor([[0, 1, 2, 3, 4, 5],
        [6, 7, 8, 9, 0, 1]])

In [ ]:
class tail_only_ComplEx(ComplEx):

    '''
    Overwritting random_sample() to make negative triples by setting a random tail to each triple,
    instead of setting a random head or tail.
    '''
    @torch.no_grad()
    def random_sample(
        self,
        head_index: torch.Tensor,
        rel_type: torch.Tensor,
        tail_index: torch.Tensor,
        ) -> torch.Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:

        """
        Randomly samples negative triplets by replacing the tail.
        Args:
            head_index (torch.Tensor): The head indices.
            rel_type (torch.Tensor): The relation type.
            tail_index (torch.Tensor): The tail indices.
        """

        tail_index = shuffle_tensor(tail_index.clone())

        return head_index, rel_type, tail_index

to = tail_only_ComplEx(
    num_nodes=train_data.num_nodes,
    num_relations = train_data.edge_index.size()[1],
    hidden_channels=hidden_channels,
).to(device)



print("Heads are not touched :")
print(batchy[0])
print(to.random_sample(*batchy)[0])

print("\nBut tails are :")
print(batchy[2])
print(to.random_sample(*batchy)[2])

def tens_to_set(t):
    return set(t.tolist())

t, f = batchy[2], to.random_sample(*batchy)[2]
st, sf = tens_to_set(t), tens_to_set(f)
print('Shuffle ok :', not bool(len(st - sf)))

Heads are not touched :
tensor([12043,  2912, 16271,  ...,   478,  9393, 27290])
tensor([12043,  2912, 16271,  ...,   478,  9393, 27290])

But tails are :
tensor([16231,  1781, 13382,  ..., 18714,  3859,   157])
tensor([25640, 14519, 19938,  ..., 11936,  3859, 16048])
Shuffle ok : True


In [ ]:
class best_LinSim_ComplEx(tail_only_ComplEx):
  def loss(
            self,
            head_index: torch.Tensor,
            rel_type: torch.Tensor,
            tail_index: torch.Tensor,
            ) -> torch.Tensor:
            
        '''
        tail_only_ComplEx.loss() modified to account a LinSim term :
        one withdraw the mean(bests similarities between each false tail of a triple to its possible tails) to the loss.
        '''

        pos = head_index, rel_type, tail_index

        false_head_index, false_rel_type, false_tail_index = self.random_sample(head_index, rel_type, tail_index)
        neg = false_head_index, false_rel_type, false_tail_index

        pos_score = self(*pos)
        neg_score = self(*neg)
        scores = torch.cat([pos_score, neg_score], dim=0)

        pos_target = torch.ones_like(pos_score) 
        neg_target = torch.zeros_like(neg_score)
        target = torch.cat([pos_target, neg_target], dim=0)

        # Calculating LinSim(positive_head, negative_head) : 
        similarities = best_lin_sims_for_batch(head_index, rel_type, false_tail_index)

        return F.binary_cross_entropy_with_logits(scores, target) - torch.mean(similarities)

class LinSim_ComplEx(tail_only_ComplEx):
  def loss(
            self,
            head_index: torch.Tensor,
            rel_type: torch.Tensor,
            tail_index: torch.Tensor,
            ) -> torch.Tensor:
            
        '''
        tail_only_ComplEx.loss() modified to account a LinSim term : one simply withdraw mean(similarities(batch)) to the loss.
        '''

        pos = head_index, rel_type, tail_index

        false_head_index, false_rel_type, false_tail_index = self.random_sample(head_index, rel_type, tail_index)
        neg = false_head_index, false_rel_type, false_tail_index

        pos_score = self(*pos)
        neg_score = self(*neg)
        scores = torch.cat([pos_score, neg_score], dim=0)

        pos_target = torch.ones_like(pos_score) 
        neg_target = torch.zeros_like(neg_score)
        target = torch.cat([pos_target, neg_target], dim=0)

        # stacking true and falses tails in df :
        pos_and_neg_tails = pd.DataFrame(torch.stack((tail_index,false_tail_index)).transpose(0,1)).astype("int")

        # Calculating LinSim(positive_head, negative_head) : 
        similarities = torch.tensor(pos_and_neg_tails.apply(lambda row : lin_sim_on_mapped_terms(row[0], row[1]),
                                                      axis = 1).values
                                    )

        return F.binary_cross_entropy_with_logits(scores, target) - torch.mean(similarities)

class LinSim_Only_ComplEx(tail_only_ComplEx):
  def loss(
            self,
            head_index: torch.Tensor,
            rel_type: torch.Tensor,
            tail_index: torch.Tensor,
            ) -> torch.Tensor:
            
        '''
        tail_only_ComplEx.loss() modified to account a LinSim term : one simply withdraw mean(similarities(batch)) to the loss.
        '''

        pos = head_index, rel_type, tail_index

        false_head_index, false_rel_type, false_tail_index = self.random_sample(head_index, rel_type, tail_index)
        neg = false_head_index, false_rel_type, false_tail_index

        pos_score = self(*pos)
        neg_score = self(*neg)
        scores = torch.cat([pos_score, neg_score], dim=0)

        pos_target = torch.ones_like(pos_score) 
        neg_target = torch.zeros_like(neg_score)
        target = torch.cat([pos_target, neg_target], dim=0)

        # stacking true and falses tails in df :
        pos_and_neg_tails = pd.DataFrame(torch.stack((tail_index,false_tail_index)).transpose(0,1)).astype("int")

        # Calculating LinSim(positive_head, negative_head) : 
        similarities = torch.tensor(pos_and_neg_tails.apply(lambda row : lin_sim_on_mapped_terms(row[0], row[1]),
                                                      axis = 1).values
                                    )

        return torch.tensor([1])- torch.mean(similarities)

In [ ]:
def getBack(var_grad_fn):
    print(var_grad_fn)
    for n in var_grad_fn.next_functions:
        if n[0]:
            try:
                tensor = getattr(n[0], 'variable')
                print(n[0])
                print('Tensor with grad found:\n', tensor)
                print(' - gradient:\n', tensor.grad)
                print()
            except AttributeError as e:
                getBack(n[0])

In [ ]:
# print('\n\n LinSim only\n')
# c = LinSim_Only_ComplEx(
#     num_nodes=train_data.num_nodes,
#     num_relations = train_data.edge_index.size()[1],
#     hidden_channels=hidden_channels,
# ).to('cpu')


# loss = c.loss(*batchy)
# loss.requires_grad = True
# print('loss :', loss)

# print('Tracing back tensors:')
# loss.backward()
# getBack(loss.grad_fn)

In [ ]:
print('\n\n Best Lin Sim\n')


blc = best_LinSim_ComplEx(
    num_nodes=train_data.num_nodes,
    num_relations = train_data.edge_index.size()[1],
    hidden_channels=hidden_channels,
).to('cpu')

loss = blc.loss(*batchy)
print(loss)

print('Tracing back tensors:')
loss.backward()
getBack(loss.grad_fn)




 Best Lin Sim

tensor(0.3287, grad_fn=<SubBackward0>)
Tracing back tensors:
Tensor with grad found:
 Parameter containing:
tensor([[-1.7307e-03, -1.6696e-03,  7.1589e-03,  ...,  9.0332e-03,
          1.2983e-02, -6.9542e-03],
        [-1.0307e-02, -2.3314e-03, -4.7562e-03,  ..., -1.3470e-02,
          1.0987e-02,  1.1428e-04],
        [-4.7732e-03, -1.7102e-05, -2.0934e-03,  ..., -1.1912e-02,
          7.4433e-03, -8.5728e-03],
        ...,
        [-1.2293e-02, -1.2867e-02,  1.1195e-02,  ..., -1.3378e-02,
         -7.9724e-03, -1.3584e-02],
        [ 5.0882e-04,  9.1016e-04, -6.4387e-05,  ..., -1.0366e-02,
          1.2420e-02,  5.5104e-04],
        [ 1.2019e-02,  2.0910e-04, -1.0429e-02,  ..., -6.3061e-03,
          1.6165e-04, -7.2988e-03]], requires_grad=True)
 - gradient:
 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0

In [ ]:
print('\n\n \n')

print('\n\n  LinSim\n')
lc = LinSim_ComplEx(
    num_nodes=train_data.num_nodes,
    num_relations = train_data.edge_index.size()[1],
    hidden_channels=hidden_channels,
).to('cpu')

loss = lc.loss(*batchy)
print(loss)

print('Tracing back tensors:')
loss.backward()
getBack(loss.grad_fn)




 



  LinSim

tensor(0.6012, dtype=torch.float64, grad_fn=<SubBackward0>)
Tracing back tensors:
Tensor with grad found:
 Parameter containing:
tensor([[ 0.0086, -0.0113,  0.0043,  ..., -0.0101,  0.0067, -0.0008],
        [-0.0138, -0.0023,  0.0073,  ..., -0.0024, -0.0126, -0.0093],
        [-0.0037,  0.0065,  0.0022,  ..., -0.0031, -0.0087,  0.0073],
        ...,
        [ 0.0075,  0.0012,  0.0037,  ..., -0.0087,  0.0104, -0.0053],
        [ 0.0046, -0.0072, -0.0053,  ..., -0.0027, -0.0096, -0.0120],
        [-0.0038, -0.0057, -0.0098,  ..., -0.0001, -0.0014, -0.0112]],
       requires_grad=True)
 - gradient:
 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

Tensor with grad found:
 Parameter containing:
tensor([[-1.7178e-03, -6.7244e-04,  7.0906e-05,  ...,  1.0515e-03,
          

# Defining training functions

In [ ]:
def train(loader, model, optimizer, device):
    model.train()
    total_loss = total_examples = 0
    for head_index, rel_type, tail_index in loader:

        head_index, rel_type, tail_index = head_index.to(device), rel_type.to(device), tail_index.to(device)
        
        optimizer.zero_grad()
        loss = model.loss(head_index, rel_type, tail_index)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * head_index.numel()
        total_examples += head_index.numel()
    return total_loss / total_examples

@torch.no_grad()
def test(data, model,device):
    model.eval()
    return model.test(
        head_index=data.edge_index[0].to(device),
        tail_index=data.edge_index[1].to(device),
        rel_type=data.edge_attr.to(device),
        batch_size=batch_size,
        k=10, #The k in Hit@k
    )

def get_test_loss(loader, model, device):
    model.eval()
    total_loss = total_examples = 0
    for head_index, rel_type, tail_index in loader:

        head_index, rel_type, tail_index = head_index.to(device), rel_type.to(device), tail_index.to(device)
        loss = model.loss(head_index, rel_type, tail_index)
        total_loss += float(loss) * head_index.numel()
        total_examples += head_index.numel()

    return total_loss / total_examples


def train_and_test_complex(
                           model,
                           train_data: torch_geometric.data.data.Data,
                           test_data : torch_geometric.data.data.Data,
                           xp_name = '',
                           epochs: int = 1000,
                           batch_size: int = 4096,
                           eval_period = 500,
                           reset_parameters = False, save_params = True,
                           use_wandb = False,
                           params_save_path = '',
                           device = 'cpu',
                           dataset_name = 'iric'
                           ):
    
    
    # ----------------------------------------------- Reset parameters
    if reset_parameters :
        model.reset_parameters()

    # ----------------------------------------------- Loader
    print('Init loader...')
    loader = model.loader(
                          head_index = train_data.edge_index[0],
                          tail_index = train_data.edge_index[1],
                          rel_type   = train_data.edge_attr,
                          batch_size = batch_size,
                          shuffle    = True)
    test_loader = model.loader(
                          head_index = test_data.edge_index[0],
                          tail_index = test_data.edge_index[1],
                          rel_type   = test_data.edge_attr,
                          batch_size = batch_size,
                          shuffle    = True)

    # ----------------------------------------------- Optimizer
    print('Init optimizer...')
    optimizer = optim.Adam(model.parameters())

    # ----------------------------------------------- WandB
    if use_wandb:
        print('Init wandb...')
        wandb.init(
            settings=wandb.Settings(start_method="fork"),
            project=xp_name,
            
            config={
            "architecture": str(type(model)),
            "dataset": dataset_name,
            "epochs": epochs,
            'hidden_channels' : hidden_channels,
            'batch_size' : batch_size
            }
        )

    # ----------------------------------------------- Train and eval
    print('Train...')

    torch.set_grad_enabled(True)
    model.to(device)

    train_losses = []
    test_losses = []
    for epoch in range(0, epochs+1):

        loss = train(loader = loader,
                     model  = model,
                     optimizer = optimizer,
                     device = device)
        test_loss = get_test_loss(
                            loader = test_loader,
                            model  = model,
                            device = device)
        
        train_losses.append(loss)
        test_losses.append(test_loss)
        

        if use_wandb : 
            wandb.log({"loss": loss, "loss on test": test_loss})
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Loss on test : {test_loss:.4f}')

    # ----------------------------------------------- Periodic Evaluation
        if eval_period:
            if epoch%eval_period == 0:
                print('Test...')
                rank, mrr, hits = test(test_data, model=model, device=device)

                print(f'Epoch: {epoch:03d}, Val Mean Rank: {rank:.2f}',
                    f'Val MRR: {mrr:.4f}, Val Hits@10: {hits:.4f}')
                if use_wandb:
                    wandb.log({"Val Mean Rank" : rank,
                            "Val MRR" : mrr,
                            "hits@10": hits})

    # ----------------------------------------------- End WandB
    if use_wandb:
        print('End wandb...')
        wandb.finish()
        print("WandB finished.")

    # ----------------------------------------------- Model to cpu
    model.to('cpu')
    
    # ----------------------------------------------- Save model
    print('Save parameters...')
    if save_params:
        torch.save(model.state_dict(), params_save_path)
        print("Model saved at", params_save_path)

    print("End")

    return model, train_losses, train_losses

# Initiating models

In [ ]:
# TODO Calculer sur le serveur.

print(f"Device : {device}, batch size : {batch_size}, epochs : {epochs}, hidden channels : {hidden_channels}")
print("DataSets :", '\n',train_data,'\n', test_data,'\n', val_data)

print("\nModel types :\n  -ComplEx ;\n  -Tail only Complex ;\n  -ComplEx with LinSim ;\n  -ComplEx with best possible LinSim")


usual_complex = ComplEx(
    num_nodes = train_data.num_nodes,
    num_relations = train_data.edge_index.size()[1],
    hidden_channels = hidden_channels,
)

to_complex = tail_only_ComplEx(
    num_nodes=train_data.num_nodes,
    num_relations = train_data.edge_index.size()[1],
    hidden_channels=hidden_channels,
)

lin_complex = LinSim_ComplEx(
    num_nodes=train_data.num_nodes,
    num_relations = train_data.edge_index.size()[1],
    hidden_channels=hidden_channels,
)

lin_only_complex = LinSim_Only_ComplEx(
    num_nodes=train_data.num_nodes,
    num_relations = train_data.edge_index.size()[1],
    hidden_channels=hidden_channels,
)

best_lin_complex = best_LinSim_ComplEx(
    num_nodes = train_data.num_nodes,
    num_relations = train_data.edge_index.size()[1],
    hidden_channels = hidden_channels,
)

models = {
    'classic':usual_complex,
    'tail_only':to_complex,
    'additionnal_lin_term':lin_complex,
    'smaller_error_term':best_lin_complex
}

Device : cpu, batch size : 4096, epochs : 500, hidden channels : 220
DataSets : 
 Data(edge_index=[2, 135400], edge_attr=[135400], num_nodes=30396, edge_label=[135400], edge_label_index=[2, 135400]) 
 Data(edge_index=[2, 152324], edge_attr=[152324], num_nodes=30396, edge_label=[33848], edge_label_index=[2, 33848]) 
 Data(edge_index=[2, 135400], edge_attr=[135400], num_nodes=30396, edge_label=[33848], edge_label_index=[2, 33848])

Model types :
  -ComplEx ;
  -Tail only Complex ;
  -ComplEx with LinSim ;
  -ComplEx with best possible LinSim


In [ ]:
device

'cpu'

In [ ]:
classic, usual_train_losses, usual_test_losses = train_and_test_complex(model=usual_complex,
                                                                    train_data=train_data, test_data=test_data,
                                                                    xp_name='Pipeline with train losses',
                                                                    epochs=1000,
                                                                    batch_size=2000,
                                                                    eval_period=0,
                                                                    reset_parameters=True,
                                                                    save_params=False,
                                                                    use_wandb = False,
                                                                    device=device
                                                                    
                                                                    )

SyntaxError: invalid syntax (3648410938.py, line 11)